In [2]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.0/973.0 kB 5.2 MB/s eta 0:00:00a 0:00:01


In [9]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from torchvision.transforms import Compose, Resize, ToTensor
from PIL import Image
import matplotlib.pyplot as plt

In [6]:
yolo_model = YOLO("yolov8n.pt")  

midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").small_transform

Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Downloading: "https://github.com/rwightman/gen-efficientnet-pytorch/zipball/master" to /home/santhosh/.cache/torch/hub/master.zip
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_lite3-b733e338.pth" to /home/santhosh/.cache/torch/hub/checkpoints/tf_efficientnet_lite3-b733e338.pth
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v2_1/midas_v21_small_256.pt" to /home/santhosh/.cache/torch/hub/checkpoints/midas_v21_small_256.pt
100%|██████████| 81.8M/81.8M [01:23<00:00, 1.03MB/s]
Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


In [1]:
import cv2
import numpy as np
import os

roi_points = []
drawing_complete = [False]
exit_requested = [False]

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and not drawing_complete[0]:
        if len(roi_points) < 4:
            roi_points.append((x, y))
            print(f"Point {len(roi_points)}: {x}, {y}")
            if len(roi_points) == 4:
                drawing_complete[0] = True
                print("ROI drawing completed. Right-click to save and exit.")

    elif event == cv2.EVENT_RBUTTONDOWN and drawing_complete[0]:
        exit_requested[0] = True
        print("Right click detected. Exiting and saving ROI points.")

# Load first frame from video
cap = cv2.VideoCapture("traffic testvideo.mp4")
ret, frame = cap.read()

if not ret:
    print("Failed to load video.")
    exit()

cv2.namedWindow("Select ROI - Click 4 Points")
cv2.setMouseCallback("Select ROI - Click 4 Points", mouse_callback)

while True:
    display_frame = frame.copy()

    # Draw points
    for point in roi_points:
        cv2.circle(display_frame, point, 5, (0, 255, 255), -1)

    # Draw rectangle when 4 points selected
    if len(roi_points) == 4:
        cv2.polylines(display_frame, [np.array(roi_points + [roi_points[0]])], isClosed=True, color=(0, 255, 0), thickness=2)

    cv2.imshow("Select ROI - Click 4 Points", display_frame)

    if exit_requested[0]:
        polygon_array = np.array(roi_points)
        np.save("roi_points.npy", polygon_array)
        print("ROI points saved to 'roi_points.npy' at", os.path.abspath("roi_points.npy"))
        break

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to cancel anytime
        print("ROI selection canceled.")
        break

cap.release()
cv2.destroyAllWindows()


Point 1: 522, 0
Point 2: 842, 1
Point 3: 1274, 712
Point 4: 2, 712
ROI drawing completed. Right-click to save and exit.
Right click detected. Exiting and saving ROI points.
ROI points saved to 'roi_points.npy' at /mnt/c/Users/santhosh/OneDrive/Desktop/trafficmanagement/roi_points.npy


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load model
model = YOLO("yolov8n.pt")

# Vehicle COCO class IDs: car, motorcycle, bus, truck, auto (auto = rickshaw not in COCO, optional)
VEHICLE_CLASSES = [2, 3, 5, 7]  # 2: car, 3: motorcycle, 5: bus, 7: truck

# --- Load ROI from saved file ---
try:
    roi_polygon = np.load("roi_points.npy")
    roi_polygon = roi_polygon.tolist()  # Convert to list of tuples
    print(f"Loaded ROI points: {roi_polygon}")
except Exception as e:
    print("Failed to load ROI points. Make sure roi_points.npy exists.")
    exit()

# Helper function
def is_inside_polygon(point, polygon):
    return cv2.pointPolygonTest(np.array(polygon, dtype=np.int32), point, False) >= 0

# Load video
cap = cv2.VideoCapture("traffic testvideo.mp4")
ret, first_frame = cap.read()

if not ret:
    print("Failed to read video.")
    exit()

# Tracking setup
visible_ids = set()
current_count = 0

# --- Main Loop ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 tracking
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")[0]

    output_frame = frame.copy()
    current_ids = set()

    if results.boxes.id is not None:
        ids = results.boxes.id.cpu().numpy()
        boxes = results.boxes.xyxy.cpu().numpy()
        class_ids = results.boxes.cls.cpu().numpy()

        for box, track_id, cls in zip(boxes, ids, class_ids):
            cls = int(cls)
            track_id = int(track_id)
            if cls not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box)
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            if is_inside_polygon(center, roi_polygon):
                current_ids.add(track_id)

                # Draw box and center
                cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(output_frame, f'ID: {track_id}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.circle(output_frame, center, 5, (255, 0, 0), -1)

    # Count logic
    new_ids = current_ids - visible_ids
    exited_ids = visible_ids - current_ids
    current_count += len(new_ids) - len(exited_ids)
    visible_ids = current_ids

    # Draw ROI
    cv2.polylines(output_frame, [np.array(roi_polygon, dtype=np.int32)], True, (0, 255, 255), 2)
    cv2.putText(output_frame, f"Vehicles in ROI: {current_count}", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Display
    cv2.imshow("YOLOv8 ROI Vehicle Counter", output_frame)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loaded ROI points: [[500, 14], [857, 4], [1271, 398], [102, 419]]

0: 384x640 2 cars, 1 truck, 85.2ms
Speed: 4.8ms preprocess, 85.2ms inference, 12.8ms postprocess per image at shape (1, 3, 384, 640)


In [6]:
# Load MiDaS model
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas.eval()

Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/santhosh/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [10]:
# midas_with_augmentation.py
import cv2
import torch
import numpy as np
import os

# ==================== MiDaS Setup ====================
print("[INFO] Loading MiDaS model and transforms...")
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)
midas.eval()

# ==================== Load ROI ====================
roi_file = "roi_points.npy"
if not os.path.exists(roi_file):
    print("ROI file not found.")
    exit()

roi_points = np.load(roi_file)
roi_polygon = np.array(roi_points, dtype=np.int32)

# ==================== Augmentation Function ====================
def enhance_frame(frame):
    # --- Bilateral filter for denoising while preserving edges ---
    smooth = cv2.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)

    # --- Sobel edge detection ---
    gray = cv2.cvtColor(smooth, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel_edges = cv2.magnitude(sobelx, sobely)
    sobel_edges = np.clip(sobel_edges, 0, 255).astype(np.uint8)
    sobel_colored = cv2.cvtColor(sobel_edges, cv2.COLOR_GRAY2BGR)

    # --- Fuse edge map and smooth image ---
    fused = cv2.addWeighted(smooth, 0.8, sobel_colored, 0.2, 0)

    # --- Sharpening ---
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1,  5, -1],
                               [0, -1, 0]])
    final = cv2.filter2D(fused, -1, sharpen_kernel)

    return final

# ==================== Video Stream ====================
cap = cv2.VideoCapture("traffic testvideo.mp4")
if not cap.isOpened():
    print("Failed to open video.")
    exit()

print("[INFO] Processing video... Press 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # === Optional Resize to Improve Performance ===
    frame = cv2.resize(frame, (1280, 720))  # You can comment this out if needed

    # === Apply Augmentation ===
    enhanced_frame = enhance_frame(frame)

    # === MiDaS Depth Estimation ===
    input_image = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2RGB)
    input_tensor = midas_transforms(input_image).to(device)

    with torch.no_grad():
        prediction = midas(input_tensor)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=frame.shape[:2],
            mode="bicubic",
            align_corners=False
        ).squeeze()

    # === Normalize Depth Map ===
    depth_map = prediction.cpu().numpy()
    depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # === Mask ROI ===
    mask = np.zeros_like(depth_map_normalized)
    cv2.fillPoly(mask, [roi_polygon], 255)
    masked_depth = cv2.bitwise_and(depth_map_normalized, mask)

    # === Depth Histogram-Based Queue Estimation ===
    nonzero_depth = masked_depth[masked_depth > 0]

    if nonzero_depth.size > 0:
        # Histogram bins: Close (0-85), Medium (86-170), Far (171-255)
        hist, bin_edges = np.histogram(nonzero_depth, bins=[0, 85, 170, 256])
        close_count, medium_count, far_count = hist
        total = close_count + medium_count + far_count
        queue_ratio = close_count / total if total > 0 else 0
        queue_length = queue_ratio * 100  # Percentage indicator of queue length
    else:
        queue_length = 0

    # === Overlay Display ===
    overlay = cv2.cvtColor(masked_depth, cv2.COLOR_GRAY2BGR)
    cv2.polylines(overlay, [roi_polygon], isClosed=True, color=(0, 255, 0), thickness=2)
    cv2.putText(overlay, f"Queue Estimation: {queue_length:.2f}%", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # === Show Output ===
    cv2.imshow("ROI Depth & Queue Estimation with Augmentation", overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ==================== Cleanup ====================
cap.release()
cv2.destroyAllWindows()


[INFO] Loading MiDaS model and transforms...


Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/santhosh/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


[INFO] Processing video... Press 'q' to quit.


KeyboardInterrupt: 

In [8]:
import cv2
import numpy as np
import torch
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from ultralytics import YOLO

# --- Load YOLO and MiDaS ---
model = YOLO("yolov8n.pt")
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)
midas.eval()

# --- Load ROI ---
try:
    roi_polygon = np.load("roi_points.npy").tolist()
except:
    print("ROI points file not found.")
    exit()

roi_np = np.array(roi_polygon, dtype=np.int32)

# --- Video Input ---
cap = cv2.VideoCapture("traffic testvideo.mp4")
VEHICLE_CLASSES = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# --- Fuzzy Logic Setup ---
vehicle_count_input = ctrl.Antecedent(np.arange(0, 50, 1), 'vehicle_count')
queue_length_input = ctrl.Antecedent(np.arange(0, 100, 1), 'queue_length')
green_time_output = ctrl.Consequent(np.arange(10, 70, 1), 'green_time')

# Membership functions
vehicle_count_input.automf(3)  # low, medium, high
queue_length_input.automf(3)
green_time_output['short'] = fuzz.trimf(green_time_output.universe, [10, 10, 30])
green_time_output['medium'] = fuzz.trimf(green_time_output.universe, [20, 40, 60])
green_time_output['long'] = fuzz.trimf(green_time_output.universe, [40, 60, 70])

# Fuzzy Rules
rules = [
    ctrl.Rule(vehicle_count_input['poor'] & queue_length_input['poor'], green_time_output['short']),
    ctrl.Rule(vehicle_count_input['average'] | queue_length_input['average'], green_time_output['medium']),
    ctrl.Rule(vehicle_count_input['good'] | queue_length_input['good'], green_time_output['long']),
]
fuzzy_ctrl = ctrl.ControlSystem(rules)
fuzzy_sim = ctrl.ControlSystemSimulation(fuzzy_ctrl)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    overlay_frame = frame.copy()
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")[0]
    current_ids = set()
    vehicle_centers = []
    center_depths = []

    if results.boxes.id is not None:
        ids = results.boxes.id.cpu().numpy()
        boxes = results.boxes.xyxy.cpu().numpy()
        class_ids = results.boxes.cls.cpu().numpy()

        for box, track_id, cls in zip(boxes, ids, class_ids):
            if int(cls) not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box)
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            if cv2.pointPolygonTest(roi_np, center, False) >= 0:
                current_ids.add(track_id)
                vehicle_centers.append(center)
                cv2.rectangle(overlay_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(overlay_frame, f'ID: {int(track_id)}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.circle(overlay_frame, center, 4, (255, 0, 0), -1)

    vehicle_count = len(vehicle_centers)

    # === MiDaS Depth Estimation ===
    input_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = midas_transforms(input_rgb).to(device)

    with torch.no_grad():
        depth = midas(input_tensor)
        depth = torch.nn.functional.interpolate(
            depth.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False
        ).squeeze()

    depth_np = depth.cpu().numpy()
    depth_display = cv2.normalize(depth_np, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # --- Depth Estimation from Polygon Points (3rd and 4th Points) ---
    # Extract 3rd and 4th points (near the camera)
    start_point = tuple(roi_np[2])  # 3rd point (index 2)
    end_point = tuple(roi_np[3])    # 4th point (index 3)

    # Draw a line between the 3rd and 4th points (for visualization)
    cv2.line(frame, start_point, end_point, (255, 0, 0), 3)

    # Masking the depth around the region of interest
    mask = np.zeros_like(depth_display)
    cv2.fillPoly(mask, [roi_np], 255)
    masked_depth = cv2.bitwise_and(depth_display, mask)

    for center in vehicle_centers:
        x, y = center
        if 0 <= y < depth_np.shape[0] and 0 <= x < depth_np.shape[1]:
            center_depths.append(depth_np[y, x])

    # === Calculate Queue Length Only if Vehicle is Near the Camera ===
    if center_depths:
        # Set a threshold depth for vehicles near the camera
        depth_threshold = 100  # Arbitrary threshold for depth near camera
        near_vehicle_depths = [depth for depth in center_depths if depth < depth_threshold]

        if near_vehicle_depths:
            nearest = min(near_vehicle_depths)
            farthest = max(near_vehicle_depths)
            queue_length = farthest - nearest
        else:
            queue_length = 0
    else:
        queue_length = 0

    # === Fuzzy Inference ===
    fuzzy_sim.input['vehicle_count'] = min(vehicle_count, 49)
    fuzzy_sim.input['queue_length'] = min(queue_length, 99)
    fuzzy_sim.compute()
    green_signal_time = int(fuzzy_sim.output['green_time'])

    # === Overlay Display ===
    heatmap = cv2.applyColorMap(masked_depth, cv2.COLORMAP_JET)
    combined = cv2.addWeighted(overlay_frame, 0.6, heatmap, 0.4, 0)

    cv2.polylines(combined, [roi_np], True, (255, 255, 255), 2)
    cv2.putText(combined, f"Vehicles: {vehicle_count}", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(combined, f"Queue Length: {queue_length:.2f}", (30, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(combined, f"Green Signal Time: {green_signal_time}s", (30, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 180, 255), 2)

    cv2.imshow("YOLOv8 + MiDaS + Fuzzy System", combined)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/santhosh/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master



0: 384x640 2 cars, 1 truck, 88.9ms
Speed: 4.2ms preprocess, 88.9ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 18.4ms
Speed: 1.8ms preprocess, 18.4ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 17.7ms
Speed: 2.1ms preprocess, 17.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 19.9ms
Speed: 1.9ms preprocess, 19.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 21.6ms
Speed: 2.9ms preprocess, 21.6ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 2 trucks, 16.4ms
Speed: 2.2ms preprocess, 16.4ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 22.1ms
Speed: 1.9ms preprocess, 22.1ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 

In [1]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO

# --- Load Models ---
model = YOLO("yolov8n.pt")
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)
midas.eval()

# --- Load ROI ---
try:
    roi_polygon = np.load("roi_points.npy").tolist()
except:
    print("ROI points file not found.")
    exit()

roi_np = np.array(roi_polygon, dtype=np.int32)

# --- Video ---
cap = cv2.VideoCapture("traffic testvideo.mp4")
VEHICLE_CLASSES = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Weighting factors
count_weight = 2
depth_weight = 0.05

# Green signal duration bounds
min_green = 10  # seconds
max_green = 60  # seconds

def enhance_frame(frame):
    # CLAHE (Contrast enhancement)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    frame_clahe = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # Sharpening
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5,-1],
                               [0, -1, 0]])
    sharpened = cv2.filter2D(frame_clahe, -1, sharpen_kernel)
    return sharpened

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    overlay_frame = frame.copy()

    # === YOLOv8 VEHICLE DETECTION ===
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")[0]
    current_ids = set()
    vehicle_centers = []
    center_depths = []

    if results.boxes.id is not None:
        ids = results.boxes.id.cpu().numpy()
        boxes = results.boxes.xyxy.cpu().numpy()
        class_ids = results.boxes.cls.cpu().numpy()

        for box, track_id, cls in zip(boxes, ids, class_ids):
            if int(cls) not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box)
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            if cv2.pointPolygonTest(roi_np, center, False) >= 0:
                current_ids.add(track_id)
                vehicle_centers.append(center)
                cv2.rectangle(overlay_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(overlay_frame, f'ID: {int(track_id)}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.circle(overlay_frame, center, 4, (255, 0, 0), -1)

    vehicle_count = len(vehicle_centers)

    # === MiDaS DEPTH ESTIMATION ===
    enhanced_frame = enhance_frame(frame)
    input_rgb = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2RGB)
    input_tensor = midas_transforms(input_rgb).to(device)

    with torch.no_grad():
        depth = midas(input_tensor)
        depth = torch.nn.functional.interpolate(
            depth.unsqueeze(1), size=frame.shape[:2], mode="bicubic", align_corners=False
        ).squeeze()

    depth_np = depth.cpu().numpy()
    depth_display = cv2.normalize(depth_np, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Mask for ROI
    mask = np.zeros_like(depth_display)
    cv2.fillPoly(mask, [roi_np], 255)
    masked_depth = cv2.bitwise_and(depth_display, mask)

    # Collect depth values at vehicle centers
    for center in vehicle_centers:
        x, y = center
        if 0 <= y < depth_np.shape[0] and 0 <= x < depth_np.shape[1]:
            center_depths.append(depth_np[y, x])

    # === Queue Length Calculation ===
    if center_depths:
        filtered_depths = np.clip(center_depths, 0, np.percentile(center_depths, 90))
        nearest_depth = np.min(filtered_depths)
        farthest_depth = np.max(filtered_depths)
        queue_length = farthest_depth - nearest_depth
    else:
        queue_length = 0
        nearest_depth = 0
        farthest_depth = 0

    # === Density Score and Green Signal Time ===
    density_score = (vehicle_count * count_weight) + (queue_length * depth_weight)
    norm_score = min(density_score / 100, 1.0)
    green_time = int(min_green + (max_green - min_green) * norm_score)

    # === Overlay Heatmap ===
    heatmap = cv2.applyColorMap(masked_depth, cv2.COLORMAP_JET)
    combined = cv2.addWeighted(overlay_frame, 0.6, heatmap, 0.4, 0)

    # === Display Stats ===
    cv2.polylines(combined, [roi_np], True, (255, 255, 255), 2)
    cv2.putText(combined, f"Vehicles in ROI: {vehicle_count}", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(combined, f"Queue Length: {queue_length:.2f}", (30, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(combined, f"Density Score: {density_score:.2f}", (30, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 180, 255), 2)
    cv2.putText(combined, f"Nearest Depth: {nearest_depth:.2f}", (30, 160),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 255, 200), 2)
    cv2.putText(combined, f"Farthest Depth: {farthest_depth:.2f}", (30, 200),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 200, 200), 2)
    cv2.putText(combined, f"Green Time: {green_time} sec", (30, 240),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 180), 2)

    cv2.imshow("YOLOv8 + MiDaS Heatmap Overlay", combined)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


/home/santhosh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master
/home/santhosh/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/santhosh/.local/lib/python3.10/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(
Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master



0: 384x640 2 cars, 1 truck, 32.8ms
Speed: 8.4ms preprocess, 32.8ms inference, 18.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 26.3ms
Speed: 3.7ms preprocess, 26.3ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 8.6ms
Speed: 1.4ms preprocess, 8.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 11.9ms
Speed: 1.7ms preprocess, 11.9ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 2 trucks, 14.7ms
Speed: 1.9ms preprocess, 14.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 8.9ms
Speed: 1.4ms preprocess, 8.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 9.0ms

KeyboardInterrupt: 

In [7]:
import cv2
import torch
import numpy as np

# Load YOLOv8 model
yolo_model =  YOLO("yolov8n.pt")
# Load MiDaS model for depth estimation
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

# Load image (this will be your input frame)
image = cv2.imread('input_image.jpg')

# 1. Perform YOLOv8 detection
results = yolo_model(image)

# Access the results
boxes = results[0].boxes  # This should give you the bounding boxes

# For each detected box, the output should be [x1, y1, x2, y2, confidence, class_id]
boxes_data = boxes.xyxy  # Bounding box coordinates in xyxy format

# You can also access labels and confidence if needed
labels = results[0].names  # Object labels

# 2. Apply MiDaS depth estimation
depth_transform = midas_transforms.Compose([
    midas_transforms.Resize(384),
    midas_transforms.ToTensor(),
    midas_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_image = depth_transform(image).unsqueeze(0)
with torch.no_grad():
    depth_map = midas(input_image)

# Convert depth map to numpy for easier handling
depth_map = depth_map.squeeze().cpu().numpy()

# 3. For each detected vehicle (bounding box), check the depth
for box in boxes_data:
    x1, y1, x2, y2 = map(int, box[:4])  # Extract bounding box coordinates (x1, y1, x2, y2)
    # Extract the depth of the area within the bounding box
    depth_values = depth_map[y1:y2, x1:x2]

    # Calculate the average depth in the bounding box region
    avg_depth = np.mean(depth_values)

    # Determine if the vehicle is closer or farther based on depth
    if avg_depth < threshold:  # Assume threshold is set based on your depth range
        print(f"Vehicle at ({x1}, {y1}) is near.")
    else:
        print(f"Vehicle at ({x1}, {y1}) is far.")

# Optionally visualize the bounding boxes and depth info
for box in boxes_data:
    x1, y1, x2, y2 = map(int, box[:4])
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw box around detected vehicle

# Show the result
cv2.imshow('Output', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/santhosh/.cache/torch/hub/facebookresearch_WSL-Images_main


WARNING ⚠️ 'source' is missing. Using 'source=/home/santhosh/.local/lib/python3.10/site-packages/ultralytics/assets'.


Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master
[ WARN:0@1147.250] global loadsave.cpp:268 findDecoder imread_('input_image.jpg'): can't open/read file: check file path/integrity



image 1/2 /home/santhosh/.local/lib/python3.10/site-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 89.7ms
image 2/2 /home/santhosh/.local/lib/python3.10/site-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 1 tie, 35.4ms
Speed: 5.7ms preprocess, 62.6ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: module 'midas.transforms' has no attribute 'Compose'

In [3]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
yolo_model = YOLO("yolov8n.pt")

# Load MiDaS and transform
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas_transform = torch.hub.load("intel-isl/MiDaS", "transforms").small_transform
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# Define vehicle classes (COCO IDs)
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Open video
cap = cv2.VideoCapture("traffic testvideo.mp4")  # Replace with 0 for webcam

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize for faster processing (optional)
    original_frame = frame.copy()
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Step 1: YOLOv8 Detection
    results = yolo_model(img_rgb)[0]
    boxes = results.boxes.data.cpu().numpy()

    vehicle_boxes = [box for box in boxes if int(box[5]) in vehicle_classes]

    # Step 2: MiDaS Depth Estimation
    input_batch = midas_transform(img_rgb).to(device).unsqueeze(0)
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=frame.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    depth_map = prediction.cpu().numpy()

    # Step 3: Calculate depth of each vehicle (bottom center point)
    queue_depths = []
    for box in vehicle_boxes:
        x1, y1, x2, y2 = map(int, box[:4])
        cx = int((x1 + x2) / 2)
        cy = y2  # bottom-center
        if 0 <= cx < depth_map.shape[1] and 0 <= cy < depth_map.shape[0]:
            depth = depth_map[cy, cx]
            queue_depths.append(depth)

            # Draw bounding box
            cv2.rectangle(original_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(original_frame, f"{depth:.2f}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Step 4: Estimate queue length
    if queue_depths:
        max_depth = max(queue_depths)
        min_depth = min(queue_depths)
        queue_length = max_depth - min_depth  # Relative queue depth

        # Display queue length
        cv2.putText(original_frame, f"Queue Length (relative): {queue_length:.2f}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(original_frame, "No vehicles detected", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show result
    cv2.imshow("Queue Length Estimation", original_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /home/santhosh/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /home/santhosh/.cache/torch/hub/intel-isl_MiDaS_master



0: 384x640 1 bus, 2 trucks, 84.5ms
Speed: 9.6ms preprocess, 84.5ms inference, 44.0ms postprocess per image at shape (1, 3, 384, 640)


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 1, 3, 129, 257]